In [2]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.svm import SVC,SVR
from src.MFTreeSearchCV import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn.datasets import load_digits,load_boston,fetch_20newsgroups
data = load_boston()

In [4]:
newsgroups_train = fetch_20newsgroups(subset='all')
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(newsgroups_train.data)
labels = newsgroups_train.target
#features =features.todense()

In [5]:
X = features
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

(15076, 173762)

In [6]:
estimator = xgb.XGBClassifier(max_depth=3,n_estimators=100,colsample_bytree=0.8,nthread=16)
param_dict = {'max_depth':{'range':[2,6],'scale':'linear','type':'int'},
'n_estimators':{'range':[50,300],'scale':'linear','type':'int'},\
'colsample_bytree':{'range':[0.3,0.9],'scale':'linear','type':'real'},
'learning_rate':{'range':[0.03,0.1],'scale':'log','type':'real'}}
fidelity_range = [500,15076]
n_jobs = 3
cv = 5
fixed_params = {'nthread':16}
scoring = 'accuracy'
              

In [8]:
t1 = time.time()
#estimator = estimator.fit(X_train,y_train)
t2 = time.time()
unit_cost = cv*(t2 - t1)
total_budget = 1000

In [9]:
model = MFTreeSearchCV(estimator=estimator,param_dict=param_dict,scoring=scoring,\
                      fidelity_range=fidelity_range,unit_cost=None,\
                    cv=5,  n_jobs = n_jobs,total_budget=total_budget,debug = True,fixed_params=fixed_params)

In [10]:
model.fixed_params

{'nthread': 16}

In [11]:
m = model.fit(X_train,y_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
m.best_params_

NameError: name 'm' is not defined

In [ ]:
y_pred = m.predict(X_test)


# newm = xgb.XGBRegressor()
# newm = newm.set_params(**m.best_params_)
# newm = newm.fit(X_train,y_train)
# y_pred = newm.predict(X_test)

In [ ]:
r2_score(y_pred,y_test)

In [ ]:
m.MP.unit_cost

In [ ]:
m.best_params_

In [ ]:
m.points
#m.evals

In [28]:
t2 -t1

58.51371097564697